In [156]:
%reload_ext autoreload
%autoreload 2
import wandb
import pandas as pd 
import torch
import numpy as np
import random 

import src.const as const
import src.env.env_basic as env_basic
import src.agents.agent_greedy as agent_greedy
import src.agents.agent_nerdy as agent_nerdy
import src.agents.agent_ppo as agent_ppo
import src.simulation.water_demands as wd 

## agent nerdy 

AttributeError: 'NoneType' object has no attribute 'name'

In [146]:
seed = 142
num_leaks = 12
df = wd.load(seed, num_leaks)

In [63]:
args = {
    'seed': seed,
    'num_leaks': num_leaks,
}


# test on last 30 days
env = env_basic.EnvLowR(df, train=False,obs_len=10)
agent = agent_nerdy.AgentAlways()

_obs = env.reset()
done = False

sum_rewards = 0
while not done:
    action = agent.get_action()
    _obs, reward, done, info, _ = env.step(action)

    # log to wandb
    sum_rewards += reward
    run.log({'tindex': info['step']-17520, 'taction': action, "treward": reward, 'tsum_rewards': sum_rewards})
run.finish()

taction,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
tindex,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
treward,▁█▁▁▁▁▁█▁▁▁█▁▁▁▁▁▁▁▁▁█▁█▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁
tsum_rewards,████▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
taction,1
tindex,2879
treward,-5
tsum_rewards,-11955


In [147]:
args = {
    'seed': seed,
    'num_leaks': num_leaks,
}

run = wandb.init(name=f'agent_never',project= 'cege_test', config=args, monitor_gym=True)

# test on last 30 days
env = env_basic.EnvLowR(df, train=False)
agent = agent_nerdy.AgentNever()

_obs = env.reset()
done = False

sum_rewards = 0
while not done:
    action = agent.get_action()
    _obs, reward, done, info,_ = env.step(action)

    # log to wandb
    sum_rewards += reward
    run.log({'tindex': info['step']-17520, 'taction': action, "treward": reward, 'tsum_rewards': sum_rewards})
run.finish()

taction,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
tindex,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
treward,████████████████████▁█████████▁█████████
tsum_rewards,▁▂▂▂▃▃▃▄▄▄▅▅▆▆▆▇▇▇██▇▃▄▄▄▅▅▅▆▆▁▁▁▂▂▂▃▃▃▄
taction,0
tindex,2879
treward,0.1
tsum_rewards,55.7


## agent greedy

In [18]:
seed = 41
num_leaks = 12
df = wd.load(seed, num_leaks)
epochs = 10

In [19]:
args = {
    'seed': seed,
    'num_leaks': num_leaks,
    'eps': 0,
    'epochs': epochs
}


for eps in [0.01, 0.05, 0.1, 0.2, 0.4]:
    args['eps'] = eps

    run = wandb.init(name=f'agent_greedy_eps{eps}_epoch{epochs}',project= 'cege_test', config=args, monitor_gym=True)

    env = env_basic.WaterLeakEnv(df, train=True)
    agent = agent_greedy.AgentGreedy()
    # train 
    sum_rewards = 0
    for i in range(epochs):
        _obs = env.reset()
        done = False
        while not done:
            action = agent.update(eps=eps)
            _obs, reward, done, info = env.step(action)
            agent.update_record(action, reward)

            # log to wandb
            sum_rewards += reward
            run.log({'index': info['step'], 'action': action, "reward": reward, 'sum_rewards': sum_rewards})
        # dump record each epoch
        run.log({'record': agent.record})
    # memory dump
    agent.dump_record()

    # test on last 30 days
    env = env_basic.WaterLeakEnv(df, train=False)
    agent = agent_greedy.AgentGreedy()
    agent.load_record() # load memory

    _obs = env.reset()
    done = False

    sum_rewards = 0
    while not done:
        action = agent.get_action(eps=eps)
        _obs, reward, done, info = env.step(action)
        
        # log to wandb
        sum_rewards += reward
        run.log({'tindex': info['step']-17520, 'taction': action, "treward": reward, 'tsum_rewards': sum_rewards})
    run.finish()

record dumped to data/train/eps_greedy_record.npy
record loaded from data/train/eps_greedy_record.npy


action,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁█▁▁█▁██▁▁▁▁▁▁█▁▁▁█▁▁█
index,▂▄▅▇▁▃▅▇▁▃▆█▂▃▅▇▁▃▅▇▂▄▆█▁▃▅▇▁▃▆█▂▄▆▇▁▃▅█
reward,███████████████▁███▁██▁█▁▁██████▁███▁██▁
sum_rewards,███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
taction,█▁▁▁▁▁▁▁▁▁█▁▁▁█▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁█▁█▁▁▁▁▁
tindex,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
treward,▁█████████▁███▁███████▁█████████▁█▁█████
tsum_rewards,██▇▇▇▇▇▇▆▇▇▇▆▆▆▇▆▆▅▅▅▅▄▄▄▄▄▄▃▃▃▃▂▂▂▂▂▂▁▁
action,0
index,17520
reward,1


In [55]:
actor_lr = 1e-3
critic_lr = 1e-2
num_episodes = 200
hidden_dim = 128
gamma = 0.98
lmbda = 0.95
epochs = 10
eps = 0.2
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

torch.manual_seed(0)

seed = 42
num_leaks = 12
df = wd.load(seed, num_leaks)

state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n


# return_list,agent = train_on_policy_agent(env, agent, num_episodes)

# load 
agent = agent_ppo.PPO(state_dim, hidden_dim, action_dim, actor_lr, critic_lr, lmbda,
            epochs, eps, gamma, device)
agent.load(str(const.PATH_DATA/'train/ppo'))
env = env_basic.WaterLeakEnv(df, train=False, obs_len=10)

state, info = env.reset()
done = False
sum_rewards = 0
while not done:
    action = agent.take_action(state)
    state, reward, done, info, _ = env.step(action)
    
    # log to wandb
    sum_rewards += reward
    # run.log({'tindex': info['step']-17520, 'taction': action, "treward": reward, 'tsum_rewards': sum_rewards})
# run.finish()


In [145]:
200/10 * i + 

180.0

## PPO

In [122]:

seed = 142
num_leaks = 12
df = wd.load(seed, num_leaks)

In [138]:
wobs = df.iloc[45:57,:].WaterDemandWithLeaks.values

In [142]:
wobs

array([5.13019989, 3.20504142, 5.55516704, 5.72289471, 0.96120466,
       0.12571989, 0.10651728, 0.17049855, 0.09836877, 0.09548576,
       0.56851048, 0.54645492])

In [141]:
multiplier = np.abs(wobs[-1]/wobs[-6:-1])
wobs[:5] = multiplier
wobsdd

array([5.13019989, 3.20504142, 5.55516704, 5.72289471, 0.96120466,
       0.12571989, 0.10651728, 0.17049855, 0.09836877, 0.09548576,
       0.56851048, 0.54645492])

In [140]:
np.abs(wobs[-1]/wobs[-6:-1])

array([5.13019989, 3.20504142, 5.55516704, 5.72289471, 0.96120466])

In [107]:
df[df.LeakageLabel==1]

,Timestamp,WaterDemandWithLeaks,LeakageLabel
112,2024-01-02 04:00:00,0.333383,1.0
113,2024-01-02 04:15:00,0.495114,1.0
114,2024-01-02 04:30:00,0.364300,1.0
115,2024-01-02 04:45:00,0.478699,1.0
116,2024-01-02 05:00:00,0.375557,1.0
...,...,...,...
20171,2024-07-29 02:45:00,-0.179265,1.0
20172,2024-07-29 03:00:00,-0.137870,1.0
20173,2024-07-29 03:15:00,-0.139828,1.0
20174,2024-07-29 03:30:00,-0.159243,1.0


In [110]:
env = env_basic.EnvBasic(df, train=False, obs_len=10)
obs = env.reset()

In [191]:
df = wd.load(243, 12)
env = env_basic.EnvLowR(df, train=False, obs_len=10)
env.train = False

import src.agents.agent_dqn as agent_dqn
agent = agent_dqn.AgentDQN(state_dim=10)
agent.load(str(const.PATH_DATA/'train/dqn'))

reward_2 = agent_dqn.test_agent(env, agent,track=False)

import src.agents.agent_nerdy as agent_nerdy
agent = agent_nerdy.AgentNever()

reward_1 = agent_dqn.test_agent(env, agent,track=False)

print(reward_1, reward_2)

166.79999999998816 189.89999999999046
